In [1]:
import Credentials      # Get API keys, etc.
import pandas as pd
from pprint import pprint as pp
import re

apikey = Credentials.apikey
baseurl = 'https://api-na.hosted.exlibrisgroup.com'
queryUpdateItem = '/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}?apikey={apikey}'

In [124]:
### Get MMS/Holding/Item IDs, Descriptions, Locations from spreadsheet ###
df = pd.read_csv("FullItemList.csv", converters={'Item ID':str, 'Holdings ID':str, 'MMS ID':str, })

In [125]:
# Strip it down to just the necessary columns
df = df[['MMS ID', 'Holdings ID', 'Item ID', 'Description', 'Permanent Location']]

# Strip leading/trailing space from Description
df.Description = df.Description.str.strip()
# Collapse multiple spaces within the Description
df.Description.replace(' +', ' ', regex=True, inplace=True)
# Remove spaces from column names
df.columns = [c.replace(' ', '') for c in df.columns]

In [126]:
# Add the fields to be filled
df[['EnumA', 'EnumB', 'ChronI', 'ChronJ']] = None
# Create a dataframe to hold JUST records that get filled
filled = pd.DataFrame()

### General steps for replacement:
- Create regex which captures info from 
- Fill Enum/Chron field with captured info from Description
- Any records with (newly) filled Enum/Chron field go into the **filled** dataframe
- Purge those same records from the **df** dataframe

#### Next step should really be to get this into a function, though:
- Arguments to pass:
  - Regex with capture groups
  - list of fields to fill
- Then just a for loop to fill each of the fields

In [ ]:
# Match descriptions with just volume & nothing else
exp = re.compile(r'^v\.(\d)+$')
df['EnumA'] = df['Description'].str.extract(exp, expand=True)[0]
filled = pd.concat([filled, df[df['EnumA'].notnull()]])
df = df[df['EnumA'].isnull()]

In [127]:
exp = re.compile(r'^v\.(\d)+$')
df['EnumA'] = df['Description'].str.extract(exp, expand=True)[0] # This [0] means just the first capture group

In [86]:
def fillAndExtract(regex, fields):
    exp = re.compile(regex)
    for i, f in enumerate(fields):
        df[f] = df['Description'].str.extract(exp, expand=True)[i]

In [122]:
fillAndExtract(r'^(v)\.(\d)+$', ['EnumA', 'EnumB'])

In [115]:
# Populate the **filled** dataframe with any records that now have Enum/Chron info
filled = pd.concat([filled, df[df[f].notnull()]])
# Purge the **df** dataframe of any records that now have Enum/Chron info
df = df[df[['EnumA','EnumB','ChronI','ChronJ']].isnull().any(1)]

In [151]:
fields = ['EnumA', 'ChronI']
df[fields].dropna(how='any')

,EnumA,ChronI
